In [5]:
import random

traits = {
    'MP:0004738': 3,  
    'MP:0002638': 4,  
    'MP:0000376': 7,
    'MP:0003449': 5,
    'MP:0000410': 10,
    'MP:0000923': 3,
    'MP:0001713': 2,
    'MP:0002276': 4,
    'MP:0001191': 6,
    'MP:0000130': 4
}

print("aver", (3+4+7+5+10+3+2+4+6+4)/10)

population_size = 100  
generations = 10  
mutation_rate = 0.01  

class Rat:
    def __init__(self):
        self.genome = random.sample(list(traits.keys()), 10)

    def fitness(self):
        return sum(traits[trait] for trait in self.genome) / len(self.genome)

def mate(rat1, rat2):
    child_genome = rat1.genome[:5] + rat2.genome[5:]
    if random.random() < mutation_rate:
        child_genome[random.randint(0, 9)] = random.choice(list(traits.keys()))
    child = Rat()
    child.genome = child_genome
    return child

population = [Rat() for _ in range(population_size)]

for _ in range(generations):
    population.sort(key=lambda rat: rat.fitness(), reverse=True)

    breeding_population = population[:population_size // 2]

    next_generation = []
    while len(next_generation) < population_size:
        parent1, parent2 = random.sample(breeding_population, 2)
        child = mate(parent1, parent2)
        next_generation.append(child)

    population = next_generation

average_fitness = sum(rat.fitness() for rat in population) / population_size
print("Average fitness of final generation:", average_fitness)


aver 4.8
Average fitness of final generation: 6.212000000000001
